In [ ]:
import time
import os
import cv2
import numpy as np
import keras
from werkzeug.wrappers import Request, Response
from flask import Flask, request, render_template
from flask import Flask

import re
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

docnet_model = keras.models.load_model('3-conv-128-filter-3-dense-0.3-DropoutPercent-32-DenseNodes-1551378358docnet_model_RealSynth_2classes_07022019lossEpoch-1-.h5')
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'
app = Flask(__name__)

img_counter = 0

DATADIR = "static"

CATEGORIES = ["upload"]

UPLOAD_FOLDER = os.path.join('static','upload')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def hello_world():
    return render_template('welcome.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files['image']
    f = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    
    # add your custom code to check that the uploaded file is a valid image and not a malicious file (out-of-scope for this post)
    file.save(f)
    
    #try:
    X_test =[]
    img_array = cv2.imread(f ,cv2.IMREAD_GRAYSCALE) 
    
    text = pytesseract.image_to_string(img_array,lang="eng")
    textog =(re.sub(r'[\W_]+', ' ', text))
    text = str.split(textog)
    c=2
                
    number=['0','0','0']
    for s in reversed(text):
        if (len(s) == 4 and s.isdigit() and c>-1):
            number[c]=s
            c=c-1
    number = ' '.join(number)
                
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    text = ''.join(filter(whitelist.__contains__, textog))
    text = str.split(text)
    text = [word for word in text if len(word) > 3 and len(word) <10]
    text = ' '.join(text)
    print(text)
    print(number)
    number="ID :"+number
    
    
    
    
    new_array = cv2.resize(img_array, (128, 76))  # resize to normalize data size
    X_test.append(new_array) 
    #except Exception as e:
    #print("general exception", e, f)

    X_test = (np.array(X_test).reshape(-1,  76,128, 1))
    #NORMALIZE
    X_test = X_test / 255

    
    
    pred = docnet_model.predict(X_test)
    if(pred[0][0] > pred [0][1]):
        prediction_confidence = "Aadhar "+np.array2string(pred[0][0]*100)+"% Confident"
    else:
        prediction_confidence = "PAN "+np.array2string(pred[0][1]*100)+"% Confident"
        
    return render_template('welcome.html', user_image = f,output=prediction_confidence,info=text,no=number)

@app.route('/capture')
def captureWebcam():
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()

        #cv2.putText(og,'Press q to Capture',(50,100), font, 2,(255,255,255),2,cv2.LINE_AA)
        cv2.imshow('Press q to Capture',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            img_name = "opencv_frame_{}.jpg".format(time.time())
            
            cv2.imwrite(os.path.join(app.config['UPLOAD_FOLDER'] , img_name), frame)
            print("{} written!".format(img_name))
            f = os.path.join(app.config['UPLOAD_FOLDER'], img_name)
            break
    cap.release()
    cv2.destroyAllWindows()
    
    X_test =[]
    img_array = cv2.imread(f ,cv2.IMREAD_GRAYSCALE) 
    
    text = pytesseract.image_to_string(img_array,lang="eng")
    textog =(re.sub(r'[\W_]+', ' ', text))
    text = str.split(textog)
    c=2
                
    number=['0','0','0']
    for s in reversed(text):
        if (len(s) == 4 and s.isdigit() and c>-1):
            number[c]=s
            c=c-1
    number = ' '.join(number)
                
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    text = ''.join(filter(whitelist.__contains__, textog))
    text = str.split(text)
    text = [word for word in text if len(word) > 3 and len(word) <10]
    text = ' '.join(text)
    print(text)
    print(number)
    number="ID :"+number
    new_array = cv2.resize(img_array, (128, 76))  # resize to normalize data size
    X_test.append(new_array) 
    #except Exception as e:
    #print("general exception", e, f)

    X_test = (np.array(X_test).reshape(-1, 76, 128,1))
    #NORMALIZE
    X_test = X_test / 255

    
    
    pred = docnet_model.predict(X_test)
    if(pred[0][0] > pred [0][1]):
        
        if(pred[0][0] < 0.99):
            
            prediction_confidence = "Aadhar "+np.array2string(pred[0][0]*100)+"% Confident"
        else:
            
            prediction_confidence = "Not confident "+np.array2string(pred[0][0]*100)+"% Confident"
    else:
         if(pred[0][1] < 0.99):
                prediction_confidence = "PAN "+np.array2string(pred[0][0]*100)+"% Confident"
         else:
            prediction_confidence = "Not Confident "+np.array2string(pred[0][1]*100)+"% Confident"
    
    return render_template('welcome.html', user_image = f,output=prediction_confidence,info= text,no=number)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)


 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2019 20:01:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 20:01:20] "GET /static/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2019 20:01:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 20:01:32] "GET /static/bootstrap.min.css HTTP/1.1" 404 -


Permanent Aswan Numbm
0 0 0


127.0.0.1 - - [14/Mar/2019 20:01:59] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 20:02:00] "GET /static/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2019 20:03:37] "POST /upload HTTP/1.1" 200 -


Manpreet Singh Malagar Singh Raulu Majra Rupnagar Punjab ERIE Your Aadhaar Year Birth Male
4532 4740 1984


127.0.0.1 - - [14/Mar/2019 20:03:38] "GET /static/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2019 20:03:38] "GET /static/upload/59f961dcc011b.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 20:04:07] "POST /upload HTTP/1.1" 200 -


...

INDIA Pmmanenl nmum Number HFOPKR Nlmo FRANK GANPAI KATI Fimers GANPAT KATIE
0 0 1211


127.0.0.1 - - [14/Mar/2019 21:14:44] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 21:14:44] "GET /static/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2019 21:14:44] "GET /static/upload/pan2.jpg HTTP/1.1" 200 -


GOVT INDIA duel Permanent Accaum Number Card HFOP FRANK GANFAT KATI Flthe GANPAT mmlsignlh
0 0 1997


127.0.0.1 - - [14/Mar/2019 21:14:52] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 21:14:52] "GET /static/upload/pan23.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 21:14:52] "GET /static/bootstrap.min.css HTTP/1.1" 404 -


GOVT INDIA duel Permanent Accaum Number Card HFOP FRANK GANFAT KATI Flthe GANPAT mmlsignlh
0 0 1997


In [7]:
import keras; print(keras.__version__)

2.2.4
